# Data Processing

This notebooks takes a database in SQLite format, containing the dataset dump from the Copyright Evidence Wiki.

Note that we had a MySQL dump. We used this script to convert it into a SQLite database.

https://www.stat.berkeley.edu/~statcur/Workshop2/Assignments/Baseball/mysql2sqlite.sh

### SQLite => Pandas DataFrames

In [1]:
import sqlite3 as lite
import pandas as pd

In [2]:
con = lite.connect('euhack.db')

In [3]:
def get_table(table):
    return pd.read_sql_query('select * from {0}'.format(table), con)

This is the list of tables. We will create a DataFrame for each of them. When needed, we will join DataFrames.

In [4]:
tables = {
          'authors': 'database_author',
          'countries': 'database_country',
          'datasets': 'database_dataset',
          'disciplines': 'database_discipline',
          'domains': 'database_domain',
          'policies': 'database_evidencebasedpolicy',
          'issues': 'database_fundamentalissue',
          'requests': 'database_googledmcarequest',
          'request_domains': 'database_googledmcarequestdomain',
          'industries': 'database_industry',
          'methods': 'database_method',
          'studies': 'database_study',
          'analysis_methods': 'database_study_analysis_methods',
          'study_authors': 'database_study_authors',
          'study_collection_methods': 'database_study_collection_methods',
          'study_disciplines': 'database_study_disciplines',
          'study_policies': 'database_study_evidence_based_policies',
          'study_issues': 'database_study_fundamental_issues',
          'study_industries': 'database_study_industries',
          'study_methods': 'database_study_methods',
          'study_countries': 'database_study_countries',
          'study_references': 'database_study_references'
          }

In [5]:
df = {k: get_table(v) for k, v in tables.items()}

In [6]:
studies = (df['studies'].rename(columns={'code': 'id', 'label': 'authors', 'title': 'label'})
           .set_index('id')
           .assign(type='Study'))

Many tables include the _study id_ and a different _id_. For instance, the countries table: 

In [7]:
countries = df['study_countries']
countries.head()

,id,study_id,country_id
0,20280,"Buxmann,_Pohl,_Johnscher_and_Strube_(2005)",Germany
1,20281,Buss_and_Peukert_(2015)_2,Germany
2,20289,Deazley_(2008),United Kingdom
3,20291,De_Wolf_&_Partners_(2014),Japan
4,20292,De_Wolf_&_Partners_(2014),United Kingdom


We notice that some studies are associated to more than one country. We want to know all countries associated to a dataset. We can generalize this behavior through the following function:

In [8]:
def group_by_studies(df, column, name, src_column='study_id'):
    return (pd.DataFrame(df.groupby(src_column)
                                 .apply(lambda x: list(x[column].values)))
                                 .rename(columns={0: name}))

In [9]:
group_by_studies(countries, 'country_id', 'country_id').sample(3)

,country_id
study_id,
"Bhattacharjee,_Gopal_and_Sanders_(2003)",[United States of America]
Morris_and_Higgins_(2009),[United States of America]
"Kinnally,_Lacayo,_Mcclung_and_Sapolsky_(2008)",[United States of America]


Why we do this in this way? Because in Exhibit we need an array of values for some facets. 

In [10]:
studies = (studies
            .join(group_by_studies(df['study_industries'], 'industry_id', 'industries'))
            .join(group_by_studies(df['study_collection_methods'], 'method_id', 'collection_methods'))
            .join(group_by_studies(df['analysis_methods'], 'method_id', 'analysis_methods'))
            .join(group_by_studies(df['study_countries'], 'country_id', 'country')))

We also need other values. We compute them below.

In [11]:
citation_count = (df['study_references'].groupby('to_study_id')
                  .aggregate({'from_study_id': 'count'})
                  .rename(index={'to_study_id': 'study_id'}, columns={'from_study_id': 'citation_count'}))
citation_count.sample(3)

,citation_count
to_study_id,
Michel_(2005),1
Heald_(2014)_2,1
Liebowitz_and_Margolis_(2005),5


In [12]:
issues = df['study_issues'].join(df['issues'].set_index('code'), on='fundamentalissue_id')
issues.sample(3)

,id,study_id,fundamentalissue_id,label,description
706,14197,Sag_(2013),4,Effects of protection on industry structure,oligopolies; competition; economics of superst...
239,13730,Envisional_(2011),4,Effects of protection on industry structure,oligopolies; competition; economics of superst...
466,13957,Lunney_(2012),1,Relationship between protection,subject matter/term/scope) and supply/economic...


In [13]:
policies = df['study_policies'].join(df['policies'].set_index('code'), on='evidencebasedpolicy_id')
policies.sample(3)

,id,study_id,evidencebasedpolicy_id,label,description
216,12387,Envisional_(2011),F,Enforcement,quantifying infringement; criminal sanctions; ...
0,12171,Altschuller_and_Benbunan-Fich_(2009),F,Enforcement,quantifying infringement; criminal sanctions; ...
51,12222,Condry_(2004),F,Enforcement,quantifying infringement; criminal sanctions; ...


In [14]:
disciplines = df['study_disciplines'].join(df['disciplines'].set_index('code'), on='discipline_id')
disciplines.sample(3)

,id,study_id,discipline_id,label
241,30368,"Hanke,_Girard_and_Mattes_(2015)",O34,Intellectual Property and Intellectual Capital
951,31078,Larsson_et_al._(2014),K11,Property Law
225,30352,Handke_(2011)_2,L82,Entertainment • Media


In [15]:
studies = (studies.join(citation_count)
            .join(group_by_studies(issues, 'label', 'issues'))
            .join(group_by_studies(policies, 'label', 'policies'))
            .join(group_by_studies(disciplines, 'label', 'disciplines')))

The result:

In [16]:
studies.sample(3)

,authors,url,year,label,abstract,plain_text_proposition,intervention_response,link,authentic_link,comparative,...,data_description,type,industries,collection_methods,analysis_methods,country,citation_count,issues,policies,disciplines
id,,,,,,,,,,,,,,,,,,,,,
Buccafusco_(2012),Buccafusco (2012),http://www.copyrightevidence.org/evidence-wiki...,2012.0,Making Sense of Intellectual Property Law,Despite considerable research suggesting that ...,* There was one experiment with three differen...,Makes a case for attribution rights not being ...,http://papers.ssrn.com/sol3/papers.cfm?abstrac...,http://scholarship.kentlaw.iit.edu/fac_schol/733/,0,...,There were three different conditions run with...,Study,"[Creative, arts and entertainment]","[Quantitative Collection Methods, Survey Resea...","[Quantitative Analysis Methods, Regression ana...",[United States of America],NaN,"[Relationship between protection, Relationship...",[Fair remuneration],"[General Economics, Role of Economics • Role o..."
"Liao,_Lin_and_Liu_(2010)","Liao, Lin and Liu (2010)",http://www.copyrightevidence.org/evidence-wiki...,2010.0,Predicting the use of pirated software: A cont...,As software piracy continues to be a threat to...,* Attitude and perceived behavioral control ar...,Despite the fact that pirated software on pers...,http://www.researchgate.net/publication/225122...,http://link.springer.com/article/10.1007%2Fs10...,0,...,A Web-based survey was conducted via an electr...,Study,[Software publishing (including video games)],[Survey Research (qualitative; e.g. consumer p...,[Descriptive statistics (counting; means repor...,[Taiwan],NaN,"[Understanding consumption/use, Effects of pro...","[Fair remuneration, Enforcement]",[Information and Internet Services • Computer ...
"Ghose,_Smith_and_Telang_(2006)","Ghose, Smith and Telang (2006)",http://www.copyrightevidence.org/evidence-wiki...,2006.0,Internet Exchanges for Used Books: An Empirica...,Information technology-enabled exchanges have ...,"In this research, the authors analyze the impa...",The implication of this finding for publishers...,http://papers.ssrn.com/sol3/Papers.cfm?abstrac...,http://papers.ssrn.com/sol3/Papers.cfm?abstrac...,0,...,The study uses a unique dataset collected from...,Study,"[Publishing of books, periodicals and other pu...","[Quantitative Collection Methods, Quantitative...","[Quantitative Analysis Methods, Descriptive st...",[United States of America],NaN,"[Relationship between protection, Harmony of i...","[Nature and Scope of exclusive rights, Licensi...",[Technological Change: Choices and Consequence...


### Text Parsing

We will parse the abstracts of each study to define the most frequent words and bi-grams in each article.

In [17]:
from nltk.corpus import stopwords
en_stop = set(stopwords.words('english'))

Here we train the bi-gram detection.

In [18]:
from nltk.tokenize import sent_tokenize
from gensim.utils import tokenize
from gensim.models import Phrases

def sent_iter():
    for row in studies.itertuples():
        try:
            for sent in sent_tokenize(row.abstract):
                tokens = list(tokenize(sent))
                #print(tokens)
                #yield wordpunct_tokenize(sent)
                yield tokens
        except Exception:
            #print(row.abstract)
            continue
        #print(sentences)
        #break

bigrams = Phrases(sent_iter(), min_count=3)        

We do not want to include less-frequent words, so we also count all words in the dataset.

In [19]:
from collections import Counter

word_count = Counter()

def is_stop(bigram):
    parts = bigram.split('_')
    if len(parts) > 1:
        parts = [int(b in en_stop) for b in bigram.split('_')]
        return sum(parts) == len(parts)
    else:
        return bigram in en_stop

for row in studies.itertuples():
    if not row.abstract:
        continue
        
    for sent in sent_tokenize(row.abstract):
        tokens = list(tokenize(sent))
        if len(tokens) > 1 and tokens[0].lower() in en_stop:
            tokens = tokens[1:]
            
        tokens = [b for b in bigrams[tokens] if not is_stop(b)]
        word_count.update(tokens)
        
word_count.most_common(25)

[('copyright', 453),
 ('music', 420),
 ('piracy', 280),
 ('study', 245),
 ('digital', 207),
 ('file_sharing', 206),
 ('research', 199),
 ('also', 188),
 ('use', 187),
 ('intellectual_property', 173),
 ('new', 166),
 ('software', 163),
 ('legal', 161),
 ('data', 158),
 ('online', 154),
 ('consumers', 150),
 ('results', 147),
 ('rights', 144),
 ('economic', 134),
 ('market', 134),
 ('model', 133),
 ('works', 132),
 ('content', 132),
 ('law', 131),
 ('copyright_law', 126)]

In [20]:
def build_wordcount(row):
    if not row.abstract:
        return None
    
    token_count = Counter()
    for sent in sent_tokenize(row.abstract):
        tokens = list(tokenize(sent))
        if len(tokens) > 1 and tokens[0].lower() in en_stop:
            tokens = tokens[1:]
            
        # we consider only those with at least 5 appearances
        tokens = [b for b in bigrams[tokens] if not is_stop(b) and word_count[b] >= 5]
        token_count.update(tokens)
    
    return token_count
    

studies['wordcount'] = studies.apply(build_wordcount, axis=1)

### Cleaning

Some articles have faulty, incomplete, or wrong meta-data. Here we discard/fix them.

In [21]:
studies.abstract.fillna(value='', inplace=True)

In [22]:
studies = studies[studies.year >= 1990].copy()

In [23]:
import json

### Entities from DBPedia Spotlight

We also ran DBPedia Spotlight to perform named entity resolution in each article. In this way we recognize several entities and concepts mentioned in the documents, even though they could be described in different ways (e.g., United Kingdom or UK). We saved these results in a JSON file before running this notebook.

In [24]:
with open('/home/egraells/resources/CopyrightEvidence/processedData/EntitiesSpotlight.json') as f:
    linked = json.load(f)

In [25]:
from cytoolz import keymap

linked = keymap(lambda x: x.replace('http://www.copyrightevidence.org/evidence-wiki/index.php/', ''), linked)

In [26]:
studies['entities'] = studies.index.map(lambda x: linked[x] if x in linked else None)

In [27]:
studies['entity_count'] = studies.index.map(lambda x: dict(Counter(linked[x])) if x in linked else None)

In [28]:
studies.sample(3)

,authors,url,year,label,abstract,plain_text_proposition,intervention_response,link,authentic_link,comparative,...,collection_methods,analysis_methods,country,citation_count,issues,policies,disciplines,wordcount,entities,entity_count
id,,,,,,,,,,,,,,,,,,,,,
"Weatherall,_Webster_and_Bently_(2009)_3","Weatherall, Webster and Bently (2009)",http://www.copyrightevidence.org/evidence-wiki...,2009.0,IP Enforcement in the UK and Beyond: A Literat...,Recent reviews and policy discussions of the v...,Recent reviews and policy discussions of the v...,* The authors state that careful thought needs...,https://www.melbourneinstitute.com/downloads/i...,https://www.melbourneinstitute.com/downloads/i...,1,...,[Survey Research (qualitative; e.g. consumer p...,"[Textual Content Analysis, Legal Analysis]","[United Kingdom, United States of America, Aus...",1.0,[Understanding consumption/use],[Enforcement],"[General Economics, Law and Economics, Illegal...","{'copyright': 1, 'use': 2, 'various': 1, 'pate...",None,None
"Huygen,_Helberger,_Poort,_Rutten_and_Van_Eijk_(2009)","Huygen, Helberger, Poort, Rutten and Van Eijk ...",http://www.copyrightevidence.org/evidence-wiki...,2009.0,Ups and downs; economic and cultural effects o...,The main aim of this study is to identify the ...,- The number of music downloaders in the Nethe...,"It is up to government, as part of its cultura...",http://dare.uva.nl/document/2/79017,http://papers.ssrn.com/sol3/Papers.cfm?abstrac...,1,...,"[Quantitative Collection Methods, Survey Resea...","[Quantitative Analysis Methods, Descriptive st...",[Netherlands],1.0,"[Effects of protection on industry structure, ...","[Licensing and Business models, Enforcement]",[Microeconomic Behavior: Underlying Principles...,"{'this_study': 1, 'as_well': 1, 'effects': 1, ...","[Netherlands, File_sharing]","{'File_sharing': 1, 'Netherlands': 1}"
Sung_(2007),Sung (2007),http://www.copyrightevidence.org/evidence-wiki...,2007.0,An economic analysis of new peer-to-peer trans...,,None,None,None,None,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,[],{}


### Export Dataset to JSON in Exhibit Format

Here we export our dataset in the Collection format needed for Simile Exhibit.

In [29]:
json_data = json.loads(studies.to_json(orient='records'))
#json_data.extend(kind_data)
#json_data.extend(industry_data)
article_data = {
    'items': json_data,
               
    'types': {
        'Study': {'plural': 'Studies'}
    },
               
    'properties': {
        'year': 'number',
        'citation_count': 'number'
    }
} 

with open('/home/egraells/resources/CopyrightEvidence/visual_exploration/datasets/copyright-evidence.json', 'w') as f:
    json.dump(article_data, f)